In [52]:
from itertools import product
import polars as pl

In [53]:
tissues = ["BAT", "BLOOD", "LIVER", "HEART", "WAT-SC", "SKM-GN", "HYPOTH"]
assays = ["ATAC", "TRNSCRPT", "METHYL"]

products = list(product(tissues, assays))

In [11]:
annotations = {
    "ATAC": pl.read_csv(
        "data/ATAC_feature_annotation.csv", schema_overrides={"chrom": pl.String}
    ).select(pl.col("feature_ID", "ensembl_gene")),
    "TRNSCRPT": pl.read_csv("data/TRNSCRPT_feature_annotation.csv").select(
        pl.col("feature_ID", "gene_id")
    ),
    "METHYL": pl.read_csv(
        "data/METHYL_feature_annotation.csv", schema_overrides={"EntrezID": pl.String}
    ).select(pl.col("feature_ID", "EntrezID")),
}

In [61]:
genomic_data = pl.read_csv("data/TRAINING_REGULATED_NORM_DATA.csv", null_values=["NA"]).filter(
    pl.col("assay").is_in(assays)
).drop("dataset")
feature_to_gene = pl.read_csv("data/FEATURE_TO_GENE.csv", null_values=["NA"])

In [62]:
data: pl.DataFrame = genomic_data.join(
    feature_to_gene.select(pl.col("feature_ID", "ensembl_gene")),
    on="feature_ID",
    how="left",
).drop(pl.selectors.by_index(0))

In [69]:
data.drop("tissue", "assay", "feature_ID").group_by("feature", "ensembl_gene").mean().sort("ensembl_gene")

feature,ensembl_gene,10027327,10043527,10502300,10411690,10185585,10046461,10024735,10871349,10700102,10561072,10337199,10023259,10046380,10025707,10734945,10641858,10045228,10953744,10106383,10971645,10025626,10044922,10027599,10950486,10486003,10306471,10044841,10026193,10901434,10026355,10729518,10095241,10044337,10589325,10044760,10381414,10503110,10046119,10220917,10680837,10027165,10335064,10587543,10934529,10044256,10672656,10043950,10026517,10314733,10204989,10045309,10422463,10833331,10761160,10025979,10026789,10139044,10417702,10424911,10027408,10046542,10025464,10026274,10043799,10046704,10059369,10152148,10315624,10027912,10677887,10619259
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""TRNSCRPT;ADRNL;ENSRNOG00000000…","""ENSRNOG00000000008""",-0.7387,-0.94139,-0.59007,-2.14966,-1.43455,-0.76078,-0.63198,-0.62297,-1.79689,-1.08953,-0.02867,-0.58012,1.2856,-1.14739,-0.66794,-0.94411,-1.2895,2.72672,-1.40085,0.263,-0.48365,null,-0.20343,-1.67123,-0.77388,-1.20951,null,-0.80985,-0.92411,-0.43588,-1.63783,-1.02324,0.764,null,-0.76695,-1.50401,-0.18138,1.08491,-3.51724,null,null,-0.94709,-0.76845,null,-2.44742,-1.35228,-1.05139,null,null,null,-1.5542,-0.85559,-1.7128,-3.51724,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""TRNSCRPT;ADRNL;ENSRNOG00000000…","""ENSRNOG00000000012""",-0.96029,-1.35582,0.02768,-0.86926,-0.61576,-1.56168,1.69249,-0.74119,-0.30582,1.73951,1.10789,-2.29582,3.03771,-2.00485,-2.91582,-1.94174,-2.90721,4.1858,0.06307,1.43275,-1.31742,null,-0.87082,-0.24669,-0.36551,-0.27441,null,0.28046,-0.92411,-1.27477,-0.63691,-2.16988,2.02577,null,-0.76695,-0.47941,3.52712,1.74053,-2.37123,null,null,-0.37653,0.57194,null,-2.44742,-1.71444,-0.93783,null,null,null,-1.78647,2.4819,-0.06152,-1.06518,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""TRNSCRPT;SKM-VL;ENSRNOG0000000…","""ENSRNOG00000000012""",3.33676,2.72366,3.34911,3.22432,3.07093,2.56057,3.01327,2.65165,3.39886,2.80459,3.19826,1.91861,2.85838,3.07432,2.81308,2.84335,3.41182,2.69129,3.17792,3.05436,2.83334,null,2.79843,3.00107,2.74596,3.75096,null,2.77907,2.97214,2.41595,2.97353,3.129,2.40007,null,3.33575,3.20343,2.51432,3.00123,2.40892,null,null,3.00997,2.32237,null,2.95545,2.87391,3.10745,null,null,null,2.77594,2.63467,2.89851,2.84704,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""TRNSCRPT;BLOOD;ENSRNOG00000000…","""ENSRNOG00000000033""",10.32644,10.51219,10.61489,10.6903,10.24134,11.42502,10.7625,10.73769,11.28948,10.79336,10.68634,10.62481,10.59316,10.71052,10.79884,10.7835,10.78103,11.80194,10.64148,10.80058,11.88018,null,9.96763,10.93355,10.98993,11.07955,null,11.02782,11.068,10.35532,10.83963,10.73762,11.3904,null,11.27262,10.49174,10.7671,10.66741,10.45913,null,null,10.7103,11.06256,null,10.93289,11.03946,10.82259,null,null,null,11.0404,10.66166,11.17609,11.30269,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""TRNSCRPT;SPLEEN;ENSRNOG0000000…","""ENSRNOG00000000034""",4.92402,5.25481,4.91776,4.95928,4.98145,4.93014,5.14171,4.76962,4.94224,4.89234,4.828,5.01863,4.96145,5.00147,5.05976,4.82992,4.87617,4.964,4.88907,4.75455,5.11117,null,4.77346,4.85299,4.90152,4.83097,null,5.14089,4.89248,4.83576,4.93872,4.94595,5.06155,null,4.67974,4.87997,4.89266,4.89037,5.09387,null,null,4.91719,4.7266,null,5.08951,4.83033,4.83019,null,null,null,4.83834,4.84122,4.91153,4.90224,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TRNSCRPT;CORTEX;ENSRNOG0000006…","""ENSRNOG00000062298""

## Phenotype Processing

In [1]:
import pandas as pd

In [2]:
# Reading phenotype data
pheno = pd.read_csv("phenotype.csv")

/var/folders/lt/nn7tpp3967v9_lrhrs65k5j80000gn/T/ipykernel_9709/501660619.py:2: DtypeWarning: Columns (37,40,41,42,44,50,66,81,82,104,112,120,128,137,147,156,165,174,183,184,186,193,194,196,201,202,204,209,210,212,217,218,220,226,227,229,236,237,239,244,245,247,252,253,255,260,261,263,269,270,272,279,280,282,287,288,290,295,296,298,303,304,306,312,313,315,322,323,325,330,331,333,338,339,341,346,347,349,355,356,358,365,366,368,373,374,376,381,382,384,389,390,392,398,399,401,403,404,406,411,412,414,419,420,422,428,429,431,437,455,456,459,460,461,463,465,471,473,492) have mixed types. Specify dtype option on import or set low_memory=False.
  pheno = pd.read_csv("phenotype.csv")


In [3]:
# Subsetting phenotype data to include necessary columns
phenoSubset = pheno[["pid", 
                     "viallabel", 
                     "sex", 
                     "vo2.max.test.vo2_max_2", 
                     "registration.weight", 
                     "terminal.weight.bw", 
                     "group"]]

In [4]:
# Weight loss calculation
phenoSubset["mass_lost"] = phenoSubset["registration.weight"] - phenoSubset["terminal.weight.bw"]

phenoSubset["norm_mass_lost"] = phenoSubset["mass_lost"] / phenoSubset["registration.weight"]

/var/folders/lt/nn7tpp3967v9_lrhrs65k5j80000gn/T/ipykernel_9709/3632688668.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phenoSubset["mass_lost"] = phenoSubset["registration.weight"] - phenoSubset["terminal.weight.bw"]
/var/folders/lt/nn7tpp3967v9_lrhrs65k5j80000gn/T/ipykernel_9709/3632688668.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phenoSubset["norm_mass_lost"] = phenoSubset["mass_lost"] / phenoSubset["registration.weight"]


In [5]:
phenoSubset.head()

,pid,viallabel,sex,vo2.max.test.vo2_max_2,registration.weight,terminal.weight.bw,group,mass_lost,norm_mass_lost
0,10023259,90217013001,male,63.47,335.0,346.2,control,-11.2,-0.033433
1,10023259,90217013104,male,63.47,335.0,346.2,control,-11.2,-0.033433
2,10023259,90217013105,male,63.47,335.0,346.2,control,-11.2,-0.033433
3,10023259,90217013106,male,63.47,335.0,346.2,control,-11.2,-0.033433
4,10023259,90217013107,male,63.47,335.0,346.2,control,-11.2,-0.033433


In [ ]:
phenoSubset.to_csv("phenoSubset.csv")